# Importing Libraries and predictions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import jax.numpy as jnp
import os
os.chdir('../../')
from utilities import plot,errors
from datetime import datetime

import tensorflow_probability.substrates.jax as tfp
import jax.numpy as jnp
import jax
dist = tfp.distributions
from utilities import recalibration
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

In [2]:
from probml_utils import latexify,savefig, is_latexify_enabled

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']=str(3)

In [4]:
%env LATEXIFY=1
%env FIG_DIR=/home/desai.aadesh/temp/NILM_Uncertainty/figures

env: LATEXIFY=1
env: FIG_DIR=/home/desai.aadesh/temp/NILM_Uncertainty/figures


In [5]:
os.chdir('notebooks/dishwasher')

In [6]:
s2p_mlp_training = pd.read_csv('s2p/mlp/training_predictions.csv')
s2p_mlp_testing = pd.read_csv('s2p/mlp/testing_predictions.csv')
s2p_gmlp_training = pd.read_csv('s2p/gmlp/training_predictions.csv')
s2p_gmlp_testing = pd.read_csv('s2p/gmlp/testing_predictions.csv')

In [7]:
lstm_mlp_training = pd.read_csv('lstm/mlp/training_predictions.csv')
lstm_mlp_testing = pd.read_csv('lstm/mlp/testing_predictions.csv')
lstm_gmlp_training = pd.read_csv('lstm/gmlp/training_predictions.csv')
lstm_gmlp_testing = pd.read_csv('lstm/gmlp/testing_predictions.csv')

In [8]:
s2p_mlp_recal = pd.read_csv('s2p/mlp/recalibration_df.csv')
s2p_gmlp_recal = pd.read_csv('s2p/gmlp/recalibration_df.csv')
lstm_mlp_recal=pd.read_csv('lstm/mlp/recalibration_df.csv')
lstm_gmlp_recal=pd.read_csv('lstm/gmlp/recalibration_df.csv')

In [9]:
s2p_mlp_training.columns = map(str.lower, s2p_mlp_training.columns)
s2p_mlp_testing.columns = map(str.lower, s2p_mlp_testing.columns)
s2p_gmlp_training.columns = map(str.lower, s2p_gmlp_training.columns)
s2p_gmlp_testing.columns = map(str.lower, s2p_gmlp_testing.columns)

In [10]:
lstm_mlp_training.columns = map(str.lower, lstm_mlp_training.columns)
lstm_mlp_testing.columns = map(str.lower, lstm_mlp_testing.columns)
lstm_gmlp_training.columns = map(str.lower, lstm_gmlp_training.columns)
lstm_gmlp_testing.columns = map(str.lower, lstm_gmlp_testing.columns)

In [11]:
lstm_mlp_testing.head()

,timestamp,ground truth,mean,mc_mean,mc_sigma,de_mean,de_sigma,bs_mean,bs_sigma
0,2011-04-21 00:00:00-04:00,0.1250,8.150970,8.674476,0.541819,4.956738,2.725016,4.561386,3.794790
1,2011-04-21 00:01:00-04:00,0.1250,8.072636,8.755232,1.382341,4.860297,2.719957,4.470869,3.782062
2,2011-04-21 00:02:00-04:00,0.1250,7.985378,8.133344,1.554992,4.763817,2.713866,4.373968,3.766174
3,2011-04-21 00:03:00-04:00,0.0625,7.898744,7.303655,0.937014,4.661929,2.709787,4.277756,3.752521
4,2011-04-21 00:04:00-04:00,0.1250,7.804599,7.302094,0.823760,4.558229,2.709300,4.174093,3.737651


In [12]:
timestamp = lstm_mlp_testing["timestamp"]
timestamp = pd.to_datetime(timestamp).dt.strftime('%H:%M')

In [13]:
latexify(fig_height=1.5, fig_width=3.3)

In [14]:
fig, ax = plt.subplots(1)
ax.plot(lstm_mlp_testing["ground truth"], alpha=0.5,label="Ground\nTruth")
ax.plot(lstm_mlp_testing["de_mean"], label="Mean\npredicted")

ax.set_xticks(jnp.arange(0, len(timestamp), 2000), labels=timestamp.values[::2000])
ax.tick_params(axis='both',rotation=90)
ax.set_ylabel("Watt")
ax.legend(fontsize=5, bbox_to_anchor=(0.6, 1))
sns.despine()
savefig("dishwasher_lstm_de_mean")

saving image to /home/desai.aadesh/temp/NILM_Uncertainty/figures/dishwasher_lstm_de_mean_latexified.pdf
Figure size: [3.3 1.5]


In [15]:
fig, ax = plt.subplots(1)
ax.plot(lstm_mlp_testing["ground truth"],alpha=0.5, label="Ground\nTruth")
ax.plot(lstm_mlp_testing["de_mean"], label="Mean\npredicted")
ax.plot(lstm_mlp_testing["de_sigma"], color="green", label="Sigma\npredicted")

ax.set_xticks(jnp.arange(0, len(timestamp), 2000), labels=timestamp.values[::2000])
ax.tick_params(axis='both',rotation=90)
ax.set_ylabel("Watt")
ax.legend(fontsize=5, bbox_to_anchor=(0.6, 1))
sns.despine()
savefig("dishwasher_lstm_de_sigma")

saving image to /home/desai.aadesh/temp/NILM_Uncertainty/figures/dishwasher_lstm_de_sigma_latexified.pdf
Figure size: [3.3 1.5]


In [16]:
fig, ax = plt.subplots(1)
ax.plot(lstm_mlp_testing["ground truth"],alpha=0.5, label="Ground\nTruth")
ax.plot(lstm_mlp_testing["de_sigma"], color="green", label="Sigma\npredicted")

ax.set_xticks(jnp.arange(0, len(timestamp), 2000), labels=timestamp.values[::2000])
ax.tick_params(axis='both',rotation=90)
ax.set_ylabel("Watt")
ax.legend(fontsize=5, bbox_to_anchor=(0.6, 1))
sns.despine()
savefig("dishwasher_lstm_de_sigma")

saving image to /home/desai.aadesh/temp/NILM_Uncertainty/figures/dishwasher_lstm_de_sigma_latexified.pdf
Figure size: [3.3 1.5]


# Define Functions

In [17]:

def rmse_(dataframe):
    def rmse_loss(y,yhat):
      return (y-yhat)**2
    return jnp.mean(jax.vmap(rmse_loss,in_axes=(0,0))(dataframe['Ideal'].values,dataframe['Counts'].values))
def mae_(dataframe):
    def mae(y,yhat):
      return jnp.abs(y-yhat)
    return jnp.mean(jax.vmap(mae,in_axes=(0,0))(dataframe['p'].values,dataframe['p_hat'].values))
def mae1(y,yhat):
    def mae(y,yhat):
      return jnp.abs(y-yhat)
    return jnp.mean(jax.vmap(mae,in_axes=(0,0))(y,yhat))
def NLL(mean,sigma,y):
    def loss_fn(mean, sigma, y):
      d = dist.Normal(loc=mean, scale=sigma)
      return -d.log_prob(y)
    return jnp.mean(jax.vmap(loss_fn, in_axes=(0, 0, 0))(mean, sigma, y))

def plot_predictions(y_true,mean,sigma):
    idx1 = 0
    idx2 = -1
    fig, ax = plt.subplots(2, 2, figsize=(18,10))
    ax = ax.ravel()
    ax[0].plot(y_true[idx1:idx2], label="True")
    ax[1].plot(mean[idx1:idx2], label=f"$\mu$ Predicted", color="orange")
    ax[2].plot(y_true[idx1:idx2], label="True")
    ax[2].plot(mean[idx1:idx2], label=f"$\mu$ Predicted", color="orange")
    #ax[3].plot(y_true[idx1:idx2], label="True", alpha=0.7)
    ax[3].plot(sigma[idx1:idx2], label=f"$\sigma$ Predicted", color="green")
    ax[0].legend(fontsize=15, bbox_to_anchor=(0.5,1))
    ax[1].legend(fontsize=15, bbox_to_anchor=(0.5,1))
    ax[2].legend(fontsize=15, bbox_to_anchor=(0.5,1))
    ax[3].legend(fontsize=15, bbox_to_anchor=(0.5,1))
    sns.despine()


In [18]:
import scipy.stats as st
def calibrate(mean, sigma, Y):
    df = pd.DataFrame()
    df["mean"] = mean
    df["sigma"] = sigma
    df["Y"] = Y
    df["z"] = (df["Y"] - df["mean"]) / df["sigma"]
    df["perc"] = st.norm.cdf(df["z"])
    k = jnp.arange(0, 1.1, 0.1)
    counts = []
    df2 = pd.DataFrame()
    df2["Interval"] = k
    df2["Ideal"] = k
    for i in range(0, 11):
        l = df[df["perc"] < 0.5 + i * 0.05]
        l = l[l["perc"] >= 0.5 - i * 0.05]
        counts.append(len(l) / len(df))
    df2["Counts"] = counts
    return df2

In [19]:
def something2(tr_df,te_df,recal_df):
    for i,j in zip(grp,grp2):
        try:
            q = tr_df[i+'sigma']
            tr_nll.append(errors.NLL(tr_df[i+'mean'].values,tr_df[i+'sigma'].values,tr_df['ground truth'].values))
            te_nll.append(errors.NLL(te_df[i+'mean'].values,te_df[i+'sigma'].values,te_df['ground truth'].values))
            df =  recalibration.find_p_hat_(tr_df['ground truth'].values,tr_df[i+'mean'].values,tr_df[i+'sigma'].values) 
            tr_ce.append(mae_(df))
            df1 =  recalibration.find_p_hat_(te_df['ground truth'].values,te_df[i+'mean'].values,te_df[i+'sigma'].values) 
            te_ce.append(mae_(df1))
            te_rce.append(mae1(recal_df['p'].values,recal_df['new_phat'+j].values))
            
        except KeyError:
            # print(KeyError.args)
            tr_nll.append(0)
            te_nll.append(0)
            tr_ce.append(0)
            # tr_l2_ce.append(0)
            te_ce.append(0)
            te_rce.append(0)
            # te_l2_ce.append(0)
            # pass
            

def something(tr_df,te_df):
    for i in grp:
        
            tr_rmse.append((errors.rmse(tr_df['ground truth'].values,tr_df[i+'mean'].values)))
            te_rmse.append((errors.rmse(te_df['ground truth'].values,te_df[i+'mean'].values)))
            tr_mae.append((errors.mae(tr_df['ground truth'].values,tr_df[i+'mean'].values)))
            te_mae.append((errors.mae(te_df['ground truth'].values,te_df[i+'mean'].values)))



# Table

In [20]:
metric  =pd.DataFrame({'approach':[],'Tr_mae':[],'Te_mae':[],'Tr_rmse':[],'Te_rmse':[],'Tr_nll':[],'Te_nll':[],
'Tr_ce':[],'Te_ce':[],'Te_rce':[]})
row1 = {'approach':['s2p','s2p+mc','s2p+de','s2p+bs','gs2p','gs2p+mc','gs2p+de','gs2p+bs',
'lstm','lstm+mc','lstm+de','lstm+bs','glstm','glstm+mc','glstm+de','glstm+bs']}
grp = ['','mc_','de_','bs_']
grp2 = ['','_mc','_de','_bs']
tr_rmse,te_rmse = [],[]
tr_mae,te_mae = [],[]
tr_nll,te_nll = [],[]
tr_ce,te_ce,te_rce = [],[],[]
row1 = pd.DataFrame(row1)
metric = pd.concat([metric,row1],ignore_index=True)


In [21]:
something(s2p_mlp_training,s2p_mlp_testing)
something(s2p_gmlp_training,s2p_gmlp_testing)
something(lstm_mlp_training,lstm_mlp_testing)
something(lstm_gmlp_training,lstm_gmlp_testing)
something2(s2p_mlp_training,s2p_mlp_testing,s2p_mlp_recal)
something2(s2p_gmlp_training,s2p_gmlp_testing,s2p_gmlp_recal)
something2(lstm_mlp_training,lstm_mlp_testing,lstm_mlp_recal)
something2(lstm_gmlp_training,lstm_gmlp_testing,lstm_gmlp_recal)
metric['Tr_rmse']=tr_rmse
metric['Te_rmse']=te_rmse
metric['Tr_mae']=tr_mae
metric['Te_mae']=te_mae
metric['Tr_nll']=tr_nll
metric['Te_nll']=te_nll
metric['Tr_ce']=tr_ce
metric['Te_ce']=te_ce	
metric['Te_rce']=te_rce	
	

In [22]:
metric

,approach,Tr_mae,Te_mae,Tr_rmse,Te_rmse,Tr_nll,Te_nll,Tr_ce,Te_ce,Te_rce
0,s2p,1.5949644,12.850383,9.507036,88.49797,0,0,0,0,0
1,s2p+mc,1.6787405,12.976094,10.141713,88.60537,103187350000.0,655179950000000.0,0.47378793,0.48910263,0.42578924
2,s2p+de,1.0242324,12.469606,5.012813,78.47943,11.079868,17.201664,0.3636162,0.43721402,0.3740519
3,s2p+bs,1.0680155,11.494465,5.7702885,80.5317,1.8355069,2.3792608,0.2381119,0.05813046,0.21978857
4,gs2p,12.358407,9.611411,85.69681,92.05086,2.0192528,1.6614789,0.19629566,0.19819646,0.061218884
5,gs2p+mc,11.727242,9.26328,86.72218,92.7513,1.4534016,1.5501179,0.08034008,0.16303436,0.12972693
6,gs2p+de,13.266327,9.814224,87.05264,94.25544,2.0739903,2.1566098,0.06169572,0.13060367,0.06605796
7,gs2p+bs,14.054095,10.218866,89.321686,93.88664,4.029626,3.9047284,0.44755435,0.4342328,0.17348978
8,lstm,9.063481,12.292728,38.32781,68.59398,0,0,0,0,0
9,lstm+mc,9.07119,12.295071,38.404182,68.58845,438.4633,1014.723,0.4617296,0.49255827,0.39623168


# Analysis

In [23]:
plt.bar(metric['approach'],metric['Te_mae'])
# plt.bar(metric['approach'],metric['Te_l1_ce'])
plt.xticks(rotation=90)
sns.despine()

In [24]:
plt.bar(metric['approach'],metric['Te_ce'])
plt.xticks(rotation=90)
sns.despine()

MAE = s2p_gmlp_testing mc \
TRADEOFF = lstm_mlp_testing bs \
ECE = lstm_gmlp_testing mc

In [25]:
idx1=4750
idx2=5000

In [26]:
timestamp = s2p_gmlp_testing["timestamp"][idx1:idx2]

In [27]:
timestamp = pd.to_datetime(timestamp).dt.strftime('%H:%M')

In [28]:
def plot_figure(ax, y, mean, sigma, title):
    ax.plot(timestamp, y[idx1:idx2], label="Y")
    ax.plot(timestamp, mean[idx1:idx2], label="Mean")
    ax.plot(timestamp, sigma[idx1:idx2], label="Sigma")
    ax.set_xticks(jnp.arange(0, idx2-idx1, 30), fontsize=2)
    ax.tick_params(axis='x',rotation=60)
    error = errors.mae(y.values, mean.values)
    df = recalibration.find_p_hat_(y, mean, sigma)
    ax.set_title(f'{title} \nMAE = {error:.2f}, ECE = {mae_(df):.2f}\n')


In [29]:
def plot_callibration(ax, y, mean, sigma):
    off_idx = y <= 200
    on_idx = y > 200
    off_truth, off_mean, off_sigma = y[off_idx], mean[off_idx], sigma[off_idx]
    on_truth, on_mean, on_sigma = y[on_idx], mean[on_idx], sigma[on_idx]

    df = recalibration.find_p_hat_(y, mean, sigma)
    ax.plot(df["p"], df["p_hat"], "--", label="Total")
    total_error = mae_(df)

    df = recalibration.find_p_hat_(off_truth,  off_mean, off_sigma)
    ax.plot(df["p"], df["p_hat"], "--", label="Off")
    off_error = mae_(df)

    df = recalibration.find_p_hat_(on_truth,  on_mean, on_sigma)
    ax.plot(df["p"], df["p_hat"], "--", label="On")
    on_error = mae_(df)

    ax.plot([0,1],[0,1], "--", color="black", label="Ideal")
    ax.set_title(f'\nOn ECE = {on_error:.4f}\nOff ECE = {off_error:.4f}')

In [30]:
latexify(fig_width=6.6, fig_height=4)

In [ ]:
plt.figure()
fig, ax = plt.subplots(2, 3, sharey="row")

plot_figure(ax[0,0], s2p_gmlp_testing["ground truth"], s2p_gmlp_testing["mc_mean"], 
            s2p_gmlp_testing["mc_sigma"], "Hetero S2P MC")
plot_figure(ax[0,1], lstm_mlp_testing["ground truth"], lstm_mlp_testing["bs_mean"], 
            lstm_mlp_testing["bs_sigma"], "Homo LSTM BS")
plot_figure(ax[0,2], lstm_gmlp_testing["ground truth"], lstm_gmlp_testing["mc_mean"], 
            lstm_gmlp_testing["mc_sigma"], "Hetero LSTM MC")

plot_callibration(ax[1,0], s2p_gmlp_testing["ground truth"], s2p_gmlp_testing["mc_mean"], 
                  s2p_gmlp_testing["mc_sigma"])
plot_callibration(ax[1,1], lstm_mlp_testing["ground truth"], lstm_mlp_testing["bs_mean"], 
                  lstm_mlp_testing["bs_sigma"])
plot_callibration(ax[1,2], lstm_gmlp_testing["ground truth"], lstm_gmlp_testing["mc_mean"], 
                  lstm_gmlp_testing["mc_sigma"])

ax[0,0].legend(["Ground\nTruth"], loc="upper left", fontsize=6)
line = mlines.Line2D([], [], color='C1', marker="_", ls='', label='Mean', markersize=13,
                    markeredgewidth=1.5)
ax[0,1].legend(handles=[line], loc="upper left", fontsize=6)
line = mlines.Line2D([], [], color='green', marker="_", ls='', label='Sigma', markersize=13,
                    markeredgewidth=1.5)
ax[0,2].legend(handles=[line], loc="upper left", fontsize=6)

ax[0,0].set_xlabel("(a)")
ax[0,1].set_xlabel("(b)")
ax[0,2].set_xlabel("(c)")
ax[1,0].set_xlabel("p\n(d)")
ax[1,1].set_xlabel("p\n(e)")
ax[1,2].set_xlabel("p\n(f)")

ax[1,0].legend(fontsize=5, loc="upper left")
ax[0,0].set_ylabel("Power (W)")
ax[1,0].set_ylabel("$\hat{p}$")
sns.despine()
savefig("dishwasher_best_error_dishwasher")